In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException, UnexpectedAlertPresentException  
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import numpy as np

class PrecosRelativosDataScraper:
    def __init__(self, setor_financeiro, options, service,acoes,diretorio=None):
        self.setor_financeiro = setor_financeiro
        self.options = options
        self.service = service
        self.acoes = acoes
        self.diretorio =diretorio

    def navegador_get(self, acao):
        navegador = webdriver.Chrome(service=self.service, options=self.options)
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
        return navegador

    def obter_datas(self, navegador):
        while True:
            try:
                elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select')
                datas = [elemento.text for elemento in elementos_data]
                if len(datas) == 1:
                    string_de_datas = datas[0]
                    datas = string_de_datas.split('\n')
                return datas
            except StaleElementReferenceException:
                continue

    def obter_dados_tabela(self, navegador, data):
        while True:
            try:
                select_element = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
                select = Select(select_element)
                select.select_by_visible_text(data)

                tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                lista_resumo_balanco = []
                for linha in linhas:
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_resumo_balanco.append(celula.text)

                return lista_resumo_balanco
            except StaleElementReferenceException:
                continue

    def coletar_dados_financeiros(self, navegador, datas):
        dados = {
            'datas': datas,
            'preco_lucro': [],
            'preco_vpa': [],
            'preco_receita_liquida': [],
            'preco_fco': [],
            'preco_fcf':[np.nan] * len(datas),
            'preco_ativo_total':[],
            'preco_ebit':[np.nan] * len(datas),
            'preco_capital_giro':[np.nan] * len(datas),
            'preco_ncav':[np.nan] * len(datas),
            'ev_ebit':[np.nan] * len(datas),
            'ev_ebitda':[np.nan] * len(datas),
            'ev_receita_liquida':[np.nan] * len(datas),
            'ev_fco':[np.nan] * len(datas),
            'ev_fcf':[np.nan] * len(datas),
            'ev_atito_total':[np.nan] * len(datas),
            'market_cap_empresa':[],
            'enterprise_value':[np.nan] * len(datas),
            'dividend_yield':[]
            
    
        }
        for data in datas:
            while True:
                lista_resumo_balanco = self.obter_dados_tabela(navegador=navegador, data=data)
                if lista_resumo_balanco:
                    break
            dados['preco_lucro'].append(lista_resumo_balanco[1].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_vpa'].append(lista_resumo_balanco[3].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_receita_liquida'].append(lista_resumo_balanco[5].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_fco'].append(lista_resumo_balanco[7].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_ativo_total'].append(lista_resumo_balanco[9].replace('R$','').replace(',','.').replace('%','').strip())
            dados['market_cap_empresa'].append(lista_resumo_balanco[11].replace('R$','').replace(',','.').replace('%','').strip())
            dados['dividend_yield'].append(lista_resumo_balanco[-1].replace('R$','').replace(',','.').replace('%','').strip())
        
            
        df_resumo_balanco = pd.DataFrame(dados)
        
        return df_resumo_balanco
            
    def coletar_dados_nao_financeiros(self,navegador, datas):
        dados = {
            'datas': datas,
            'preco_lucro': [],
            'preco_vpa': [],
            'preco_receita_liquida': [],
            'preco_fco': [],
            'preco_fcf':[],
            'preco_ativo_total':[],
            'preco_ebit':[],
            'preco_capital_giro':[],
            'preco_ncav':[],
            'ev_ebit':[],
            'ev_ebitda':[],
            'ev_receita_liquida':[],
            'ev_fco':[],
            'ev_fcf':[],
            'ev_atito_total':[],
            'market_cap_empresa':[],
            'enterprise_value':[],
            'dividend_yield':[]
            
    
        }

        for data in datas:
            while True:
                lista_resumo_balanco = self.obter_dados_tabela(navegador=navegador, data=data)
                if lista_resumo_balanco:
                    break
            dados['preco_lucro'].append(lista_resumo_balanco[1].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_vpa'].append(lista_resumo_balanco[3].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_receita_liquida'].append(lista_resumo_balanco[5].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_fco'].append(lista_resumo_balanco[7].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_fcf'].append(lista_resumo_balanco[9].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_ativo_total'].append(lista_resumo_balanco[11].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_ebit'].append(lista_resumo_balanco[13].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_capital_giro'].append(lista_resumo_balanco[15].replace('R$','').replace(',','.').replace('%','').strip())
            dados['preco_ncav'].append(lista_resumo_balanco[17].replace('R$','').replace(',','.').replace('%','').strip())
            dados['ev_ebit'].append(lista_resumo_balanco[19].replace('R$','').replace(',','.').replace('%','').strip())
            dados['ev_ebitda'].append(lista_resumo_balanco[21].replace('R$','').replace(',','.').replace('%','').strip())
            dados['ev_receita_liquida'].append(lista_resumo_balanco[23].replace('R$','').replace(',','.').replace('%','').strip())
            dados['ev_fco'].append(lista_resumo_balanco[25].replace('R$','').replace(',','.').replace('%','').strip())
            dados['ev_fcf'].append(lista_resumo_balanco[27].replace('R$','').replace(',','.').replace('%','').strip())
            dados['ev_atito_total'].append(lista_resumo_balanco[29].replace('R$','').replace(',','.').replace('%','').strip())
            dados['market_cap_empresa'].append(lista_resumo_balanco[31].replace('R$','').replace(',','.').strip())
            dados['enterprise_value'].append(lista_resumo_balanco[33].replace('R$','').replace(',','.').strip())
            dados['dividend_yield'].append(lista_resumo_balanco[-1].replace('R$','').replace(',','.').replace('%','').strip())
            
            
        df_resumo_balanco = pd.DataFrame(dados)
            
        return df_resumo_balanco
    
    def rodar_acoes(self):
        iteracao = 0
        lista_dataframes = []
        acoes_processadas = set()
        for acao in self.acoes:
            
            navegador = self.navegador_get(acao=acao)
            
            elemento_text = navegador.find_elements(By.XPATH, '/html/body/div[1]/div[4]/div[2]') 
            
            if acao in acoes_processadas:
                print(f"Ação {acao} já processada, pulando.") 
                 
            elif 'código de negociação não encontrado.' in elemento_text[0].text:   
                print(f"Código de negociação {acao} não encontrado.")
            
            else:
                start_time = time.time()
            
                datas = self.obter_datas(navegador=navegador)

                if acao in self.setor_financeiro:
                    try:      
                        df_dados = self.coletar_dados_financeiros(navegador=navegador, datas=datas)  
                    except UnexpectedAlertPresentException:
                        df_dados = self.coletar_dados_financeiros(navegador=navegador, datas=datas)
                    colunas = df_dados.columns[(df_dados.columns == 'market_cap_empresa')]
                    for col in colunas:
                        df_dados[col] = df_dados[col].apply(self.converter_valor)
                    
                    df_dados['datas'] = df_dados['datas'].apply(self.converter_data)
                    
                    df_dados['dividend_yield'] = pd.to_numeric(df_dados['dividend_yield'], errors='coerce') / 100
                else:
                    try:
                        df_dados = self.coletar_dados_nao_financeiros(navegador=navegador, datas=datas)
                    except UnexpectedAlertPresentException:
                        df_dados = self.coletar_dados_nao_financeiros(navegador=navegador, datas=datas) 
                    colunas = ['market_cap_empresa','enterprise_value']
                    for col in colunas:
                        df_dados[col] = df_dados[col].apply(self.converter_valor)
                    
                    df_dados['datas'] = df_dados['datas'].apply(self.converter_data)
                        
                    df_dados['dividend_yield'] = pd.to_numeric(df_dados['dividend_yield'], errors='coerce') / 100
                    
                df_dados['tic'] = acao
                lista_dataframes.append(df_dados)

                iteracao += 1
                
                if iteracao % 5 == 0:
                    self.salvar_dados(lista_dataframes=lista_dataframes)
                    
                end_time = time.time()

                # Calcula o tempo decorrido em segundos
                execution_time_seconds = end_time - start_time

                # Calcula o tempo decorrido em minutos
                execution_time_minutes = execution_time_seconds / 60

                # Imprime o tempo de execução em minutos
                print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")
            
            acoes_processadas.add(acao) 
                
        return pd.concat(lista_dataframes)

    def salvar_dados(self,lista_dataframes):
        if self.diretorio is None:
            self.diretorio = 'dados/precos_relativos.csv'
        pd.concat(lista_dataframes).to_csv(self.diretorio)

    def converter_valor(self,valor):
        if isinstance(valor, str):
            if valor.endswith('T'):
                return float(valor.replace('T', '').replace(' ', '')) * 1_000_000
            elif valor.endswith('B'):
                return float(valor.replace('B', '').replace(' ', '')) * 1_000
            elif valor.endswith('M'):
                return float(valor.replace('M', '').replace(' ', '')) * 1
            elif valor.endswith('K'):
                return float(valor.replace('K', '').replace(' ', '')) * 0.001
            elif valor == 'NA':
                return float('nan')
        else:
            return valor
    
    def converter_data(self,valor):
        if 'Atual - ' in valor:
            return valor.replace('Atual - ', '')
        return valor

In [37]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',]
setor_financeiro = {'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'}


chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

scraper = PrecosRelativosDataScraper(setor_financeiro, options, service,acoes=codigos_ibovespa)

dados = scraper.rodar_acoes()


Tempo de execução da ação ABEV3: 2.14 minutos
Tempo de execução da ação AZUL4: 1.50 minutos
Tempo de execução da ação B3SA3: 2.91 minutos
Tempo de execução da ação BBAS3: 2.52 minutos
Tempo de execução da ação BBDC3: 2.68 minutos
Tempo de execução da ação BBDC4: 2.73 minutos
Tempo de execução da ação BBSE3: 1.41 minutos
Tempo de execução da ação BEEF3: 2.96 minutos
Tempo de execução da ação BPAC11: 1.55 minutos
Tempo de execução da ação BRAP4: 4.18 minutos


In [49]:
class SetorFinanceiroDataScraper:
    def __init__(self, setor_financeiro, options, service, acoes):
        self.setor_financeiro = setor_financeiro
        self.options = options
        self.service = service
        self.acoes = acoes
        self.diretorio = None
        self.dados = None
    
    def navegador_get(self, acao):
        navegador = webdriver.Chrome(service=self.service, options=self.options)
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
        return navegador

    def obter_dados_tabela(self, navegador):
        while True:
            try:
                tabela = navegador.find_element(By.ID, 'pagina_empresa_dados_basicos')
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
                lista_dados_basicos = []
                for linha in linhas:
                # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dados_basicos.append(celula.text)
                return lista_dados_basicos
            except StaleElementReferenceException:
                continue
                
    def rodar_acoes(self):
        acoes_processadas = set()
        dados = {
            'segmento_listagem':[],
            'setor':[],
            'segmento':[],
            'tic':[]
        }
    
        for acao in self.acoes:
            
            navegador = self.navegador_get(acao=acao)
            
            elemento_text = navegador.find_elements(By.XPATH, '/html/body/div[1]/div[4]/div[2]') 
            
            if acao in acoes_processadas:
                print(f"Ação {acao} já processada, pulando.") 
                 
            elif 'código de negociação não encontrado.' in elemento_text[0].text:   
                print(f"Código de negociação {acao} não encontrado.")
            
            else:
                start_time = time.time()
                try:      
                    df_dic = self.obter_dados_tabela(navegador=navegador)  
                except UnexpectedAlertPresentException:
                    df_dic = self.obter_dados_tabela(navegador=navegador)
                    
                dados['segmento_listagem'].append(df_dic[9])
                dados['setor'].append(df_dic[-7])
                dados['segmento'].append(df_dic[-3])
                dados['tic'].append(acao)
                end_time = time.time()
                
                # Calcula o tempo decorrido em segundos
                execution_time_seconds = end_time - start_time

                # Calcula o tempo decorrido em minutos
                execution_time_minutes = execution_time_seconds / 60

                # Imprime o tempo de execução em minutos
                print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")
                
            acoes_processadas.add(acao) 
            
        return pd.DataFrame(dados)
                

In [51]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',]
setor_financeiro = {'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'}


chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

scraper = SetorFinanceiroDataScraper(setor_financeiro, options, service,acoes=codigos_ibovespa)

dados = scraper.rodar_acoes()

Tempo de execução da ação ABEV3: 0.04 minutos
Tempo de execução da ação AZUL4: 0.03 minutos
Tempo de execução da ação B3SA3: 0.03 minutos
Tempo de execução da ação BBAS3: 0.02 minutos
Tempo de execução da ação BBDC3: 0.04 minutos
Tempo de execução da ação BBDC4: 0.11 minutos


In [40]:
dados

,segmento_listagem,setor,tic
0,Tradicional - BOVESPA,Consumo não Cíclico,ABEV3
1,Nível 2 de Governança Corporativa,Bens Industriais,AZUL4
2,Novo Mercado,Financeiro,B3SA3
3,Novo Mercado,Financeiro,BBAS3
4,Nível 1 de Governança Corporativa,Financeiro,BBDC3
5,Nível 1 de Governança Corporativa,Financeiro,BBDC4
6,Novo Mercado,Financeiro,BBSE3
7,Novo Mercado,Consumo não Cíclico,BEEF3
8,Nível 2 de Governança Corporativa,Financeiro,BPAC11
9,Nível 1 de Governança Corporativa,Materiais Básicos,BRAP4


In [6]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]

In [5]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3',]

In [3]:
# Uso da classe
from FundamentsStockBrazil.BalanceSheet import TabelaResumoDataScraper
setor_financeiro = {'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'}

chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

scraper = TabelaResumoDataScraper(setor_financeiro, options, service,acoes=codigos_ibovespa)

dados = scraper.rodar_acoes()

Tempo de execução da ação ABEV3: 1.12 minutos
Tempo de execução da ação AZUL4: 0.75 minutos
Tempo de execução da ação B3SA3: 1.64 minutos
Tempo de execução da ação BBAS3: 2.01 minutos
Tempo de execução da ação BBDC3: 2.17 minutos


In [25]:
dados

,datas,preco_lucro,preco_vpa,preco_receita_liquida,preco_fco,preco_fcf,preco_ativo_total,preco_ebit,preco_capital_giro,preco_ncav,ev_ebit,ev_ebitda,ev_receita_liquida,ev_fco,ev_fcf,ev_atito_total,market_cap_empresa,enterprise_value,dividend_yield,tic
0,Atual - 01/08/2024,12.85,1.94,2.29,7.13,9.01,1.29,9.61,50.02,-4.16,9.05,6.86,2.16,6.72,8.49,1.22,184.99 B,174.30 B,0.0622,ABEV3
1,30/06/2024,12.85,1.94,2.29,7.13,9.01,1.29,9.61,50.02,-4.16,9.05,6.86,2.16,6.72,8.49,1.22,184.99 B,174.30 B,0.0622,ABEV3
2,31/03/2024,13.22,2.21,2.41,7.37,9.44,1.43,10.28,-809.80,-4.04,9.80,7.38,2.30,7.03,9.00,1.37,191.77 B,182.74 B,0.0600,ABEV3
3,31/12/2023,13.67,2.51,2.49,8.02,10.51,1.49,10.63,-44.62,-3.41,9.96,7.53,2.33,7.51,9.84,1.40,198.18 B,185.63 B,0.0580,ABEV3
4,30/09/2023,13.43,2.24,2.46,8.97,11.97,1.47,10.88,54.65,-4.61,10.14,7.64,2.29,8.37,11.16,1.37,202.59 B,188.90 B,0.0593,ABEV3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,31/12/2000,11.96,1.05,0.90,NA,NA,0.15,1.82,3.57,-0.18,8.00,NA,3.94,NA,NA,0.65,1.03 B,4.88 B,0.0000,BRAP4
93,30/09/2000,-10.61,1.25,0.00,NA,NA,1.24,-10.46,-132.10,-64.64,-9.83,NA,0.00,NA,NA,1.17,1.05 B,1.05 B,0.0000,BRAP4
94,30/06/2000,-28.18,1.50,0.00,NA,NA,1.50,-27.80,-351.90,-165.19,-27.02,NA,0.00,NA,NA,1.45,1.38 B,1.38 B,0.0000,BRAP4
95,31/03/2000,387.61,1.52,0.00,NA,NA,1.52,387.61,0.00,0.00,367.76,NA,0.00,NA,NA,1.44,1.44 B,1.44 B,0.0000,BRAP4


In [5]:
from selenium.webdriver.chrome.service import Service
refazer_comeco = True
chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]